In [1]:
import pandas as pd
import numpy as np
import scipy as sp


In [2]:
df_train_data = pd.read_csv('../data/train/orderFuture_train.csv')
df_test_data = pd.read_csv('../data/test/orderFuture_test.csv')


In [3]:
# 首先选择action信息
df_train_action = pd.read_csv('../data/train/action_train.csv')
df_test_action = pd.read_csv('../data/test/action_test.csv')


In [4]:
df_train_action.info()
df_train_action.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1334856 entries, 0 to 1334855
Data columns (total 3 columns):
userid        1334856 non-null int64
actionType    1334856 non-null int64
actionTime    1334856 non-null int64
dtypes: int64(3)
memory usage: 30.6 MB


,userid,actionType,actionTime
0,100000000013,1,1474300753
1,100000000013,5,1474300763
2,100000000013,6,1474300874
3,100000000013,5,1474300911
4,100000000013,6,1474300936


In [5]:
# actionType对应的特征，[倒数后5个Type, 前2个actionType]
column = ['userid','last1Type','last2Type','last3Type','last4Type','last5Type',
          'last6Type','last7Type','last8Type','last9Type','last10Type',]

df_train_actionType = pd.DataFrame(columns=column)

userlist = list(set(df_train_action['userid']))
k = 0
for user in userlist:
    k+=1
    if k%1000 == 0:
        print(k)
    
    df_users = df_train_action[df_train_action['userid'] == user]
    df_users = df_users.sort_values('actionTime')
    
    row = [user]
    tmp_type = []
    tmp_type.extend(df_users['actionType'].values[-10:])
    if len(tmp_type) < 10:
        tmp_type.extend([0 for i in range(10-len(tmp_type))])
                
    row.extend(tmp_type)    
    
    df_train_actionType = df_train_actionType.append(pd.DataFrame([row],columns=column),ignore_index=True)
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000


In [7]:
# actionType对应的特征，[倒数后5个Type, 前2个actionType]
column = ['userid','last1Type','last2Type','last3Type','last4Type','last5Type',
          'last6Type','last7Type','last8Type','last9Type','last10Type',]

df_test_actionType = pd.DataFrame(columns=column)

userlist = list(set(df_test_action['userid']))
k = 0
for user in userlist:
    k+=1
    if k%1000 == 0:
        print(k)
    
    df_users = df_test_action[df_test_action['userid'] == user]
    df_users = df_users.sort_values('actionTime')
    
    row = [user]
    tmp_type = []
    tmp_type.extend(df_users['actionType'].values[-10:])
    if len(tmp_type) < 10:
        tmp_type.extend([0 for i in range(10-len(tmp_type))])
                
    row.extend(tmp_type)    
    
    df_test_actionType = df_test_actionType.append(pd.DataFrame([row],columns=column),ignore_index=True)
    

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [8]:
df_train_data = pd.merge(df_train_data,df_train_actionType, how='left',on='userid')

df_test_data = pd.merge(df_test_data,df_test_actionType, how='left',on='userid')


In [9]:
df_train_data.info()
df_test_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40307 entries, 0 to 40306
Data columns (total 12 columns):
userid        40307 non-null object
orderType     40307 non-null int64
last1Type     40307 non-null object
last2Type     40307 non-null object
last3Type     40307 non-null object
last4Type     40307 non-null object
last5Type     40307 non-null object
last6Type     40307 non-null object
last7Type     40307 non-null object
last8Type     40307 non-null object
last9Type     40307 non-null object
last10Type    40307 non-null object
dtypes: int64(1), object(11)
memory usage: 4.0+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10076 entries, 0 to 10075
Data columns (total 11 columns):
userid        10076 non-null object
last1Type     10076 non-null object
last2Type     10076 non-null object
last3Type     10076 non-null object
last4Type     10076 non-null object
last5Type     10076 non-null object
last6Type     10076 non-null object
last7Type     10076 non-null object
last8Type   

In [12]:
################## 垃圾特征------------------------------------
#
# 最近5次的type是否包含5
df_train_data['last10IncludeType5'] = df_train_data[column[1:11]].isin([5]).sum(axis=1) > 0 

# 最近5次的type是否包含6
df_train_data['last10IncludeType6'] = df_train_data[column[1:11]].isin([6]).sum(axis=1) > 0 

# 最近5次的type是否包含7
df_train_data['last10IncludeType7'] = df_train_data[column[1:11]].isin([7]).sum(axis=1) > 0 

# 最近5次的type是否包含8
df_train_data['last10IncludeType8'] = df_train_data[column[1:11]].isin([8]).sum(axis=1) > 0 

# 最近5次的type是否包含9
df_train_data['last10IncludeType9'] = df_train_data[column[1:11]].isin([9]).sum(axis=1) > 0 


# 最近5次的type是否同时包含5,6
df_train_data['last10IncludeType56'] = df_train_data['last10IncludeType5'] & df_train_data['last10IncludeType6'] 

# 最近5次的type是否同时包含5,7
df_train_data['last10IncludeType57'] = df_train_data['last10IncludeType5'] & df_train_data['last10IncludeType7'] 

# 最近5次的type是否同时包含6,7
df_train_data['last10IncludeType67'] = df_train_data['last10IncludeType6'] & df_train_data['last10IncludeType7'] 

# 最近5次的type是否同时包含5，6,7
df_train_data['last10IncludeType567'] = df_train_data['last10IncludeType5'] & df_train_data['last10IncludeType6'] & df_train_data['last10IncludeType7'] 


# 最后10次出现的5-9的种类的个数。

df_train_data['last10IncludeType59Cnt'] = df_train_data[['last10IncludeType5','last10IncludeType6',
                                                         'last10IncludeType7','last10IncludeType8',
                                                         'last10IncludeType9']].sum(axis=1)


In [13]:
################## 垃圾特征------------------------------------
#
# 最近5次的type是否包含5
df_test_data['last10IncludeType5'] = df_test_data[column[1:11]].isin([5]).sum(axis=1) > 0 

# 最近5次的type是否包含6
df_test_data['last10IncludeType6'] = df_test_data[column[1:11]].isin([6]).sum(axis=1) > 0 

# 最近5次的type是否包含7
df_test_data['last10IncludeType7'] = df_test_data[column[1:11]].isin([7]).sum(axis=1) > 0 

# 最近5次的type是否包含8
df_test_data['last10IncludeType8'] = df_test_data[column[1:11]].isin([8]).sum(axis=1) > 0 

# 最近5次的type是否包含9
df_test_data['last10IncludeType9'] = df_test_data[column[1:11]].isin([9]).sum(axis=1) > 0 


# 最近5次的type是否同时包含5,6
df_test_data['last10IncludeType56'] = df_test_data['last10IncludeType5'] & df_test_data['last10IncludeType6'] 

# 最近5次的type是否同时包含5,7
df_test_data['last10IncludeType57'] = df_test_data['last10IncludeType5'] & df_test_data['last10IncludeType7'] 

# 最近5次的type是否同时包含6,7
df_test_data['last10IncludeType67'] = df_test_data['last10IncludeType6'] & df_test_data['last10IncludeType7'] 

# 最近5次的type是否同时包含5，6,7
df_test_data['last10IncludeType567'] = df_test_data['last10IncludeType5'] & df_test_data['last10IncludeType6'] & df_test_data['last10IncludeType7'] 


# 最后10次出现的5-9的种类的个数。

df_test_data['last10IncludeType59Cnt'] = df_test_data[['last10IncludeType5','last10IncludeType6',
                                                         'last10IncludeType7','last10IncludeType8',
                                                         'last10IncludeType9']].sum(axis=1)


In [14]:
def diffInfo(x):
    if len(x) == 0:
        return x
    cur = 1514736000
    
    x = list(x)
    x.append(cur)
#     print(x)
    for i in range(len(x)-1):
        x[i] = x[i+1] - x[i]
    
    return np.array(x[:-1])

In [15]:
# 时间间隔特征，[avg,std,var,skew,kurt,min,max, last1, last2,last3,last4,last5]

column = ['userid','diffTimeAvg','diffTimeStd','diffTimeVar','diffTimeSkew','diffTimeKurt',
          'diffTimeMin','diffTimeMax','last1DiffTime','last2DiffTime','last3DiffTime',
          'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
          'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime','last13DiffTime',
          'last14DiffTime','last15DiffTime']

df_train_diffTime = pd.DataFrame(columns=column)

userlist = list(set(df_train_action['userid']))

k = 0
for user in userlist:
    k+=1
    if k%1000 == 0:
        print(k)

    df_users_t = df_train_action[df_train_action['userid'] == user]
    df_users_t = df_users_t.sort_values('actionTime')
    
    df_users_t['actionTimeDiff'] = diffInfo(df_users_t['actionTime'])
    
    row = [user]
    
    row.append(df_users_t['actionTimeDiff'].mean())
    row.append(df_users_t['actionTimeDiff'].std())
    row.append(df_users_t['actionTimeDiff'].var())
    row.append(df_users_t['actionTimeDiff'].skew())
    row.append(df_users_t['actionTimeDiff'].kurt())
    row.append(df_users_t['actionTimeDiff'].min())
    row.append(df_users_t['actionTimeDiff'].max())
    
    row.extend(df_users_t['actionTimeDiff'].values[-15:])

    if len(row) < 23:
        row.extend([-1 for i in range(23-len(row))])
    
    
    df_train_diffTime = df_train_diffTime.append(pd.DataFrame([row],columns=column),ignore_index=True)
    


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000


In [16]:
# 时间间隔特征，[avg,std,var,skew,kurt,min,max, last1, last2,last3,last4,last5]

column = ['userid','diffTimeAvg','diffTimeStd','diffTimeVar','diffTimeSkew','diffTimeKurt',
          'diffTimeMin','diffTimeMax','last1DiffTime','last2DiffTime','last3DiffTime',
          'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
          'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime','last13DiffTime',
          'last14DiffTime','last15DiffTime']

df_test_diffTime = pd.DataFrame(columns=column)

userlist = list(set(df_test_action['userid']))
k = 0
for user in userlist:
    k+=1
    if k%1000 == 0:
        print(k)
    
    df_users_t = df_test_action[df_test_action['userid'] == user]
    df_users_t = df_users_t.sort_values('actionTime')
    
    df_users_t['actionTimeDiff'] = diffInfo(df_users_t['actionTime'])
    
    row = [user]
    
    row.append(df_users_t['actionTimeDiff'].mean())
    row.append(df_users_t['actionTimeDiff'].std())
    row.append(df_users_t['actionTimeDiff'].var())
    row.append(df_users_t['actionTimeDiff'].skew())
    row.append(df_users_t['actionTimeDiff'].kurt())
    row.append(df_users_t['actionTimeDiff'].min())
    row.append(df_users_t['actionTimeDiff'].max())
    
    row.extend(df_users_t['actionTimeDiff'].values[-15:])

    if len(row) < 23:
        row.extend([-1 for i in range(23-len(row))])
    
    
    df_test_diffTime = df_test_diffTime.append(pd.DataFrame([row],columns=column),ignore_index=True)
    


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [32]:
df_test_diffTime.head()

,userid,diffTimeAvg,diffTimeStd,diffTimeVar,diffTimeSkew,diffTimeKurt,diffTimeMin,diffTimeMax,last1DiffTime,last2DiffTime,...,last6DiffTime,last7DiffTime,last8DiffTime,last9DiffTime,last10DiffTime,last11DiffTime,last12DiffTime,last13DiffTime,last14DiffTime,last15DiffTime
0,111145680897,4.131604e+05,1.546396e+06,2.391341e+12,4.810189,23.983738,2,9825870,3,7,...,15,2,29,15,17,44,40,106,296,9825870
1,114744393730,1.243792e+06,3.915845e+06,1.533385e+13,3.446476,11.910044,8,13660391,23,579325,...,518356,160962,8,18,5073,208,13660391,-1,-1,-1
2,114744393732,5.023947e+05,3.088247e+06,9.537271e+12,6.164389,37.999790,1,19038626,427,1245,...,30,13944,7158,26973,11,10,3,153,559,19038626
3,112630169606,6.882531e+05,3.076944e+06,9.467587e+12,4.472136,19.999997,2,13760741,10,233,...,3269,50,2,58,55,156,17,34,6,13760741
4,110943371277,1.753549e+06,5.762981e+06,3.321195e+13,3.573480,12.826580,12,20883876,98,86,...,29,129,30,428238,1482519,45,79,20883876,-1,-1


In [29]:
df_train_data.info()
df_test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40307 entries, 0 to 40306
Data columns (total 66 columns):
userid                    40307 non-null int64
last1Type                 40307 non-null int64
last2Type                 40307 non-null int64
last3Type                 40307 non-null int64
last4Type                 40307 non-null int64
last5Type                 40307 non-null int64
last6Type                 40307 non-null int64
last7Type                 40307 non-null int64
last8Type                 40307 non-null int64
last9Type                 40307 non-null int64
last10Type                40307 non-null int64
last10IncludeType5        40307 non-null bool
last10IncludeType6        40307 non-null bool
last10IncludeType7        40307 non-null bool
last10IncludeType8        40307 non-null bool
last10IncludeType9        40307 non-null bool
last10IncludeType56       40307 non-null bool
last10IncludeType57       40307 non-null bool
last10IncludeType67       40307 non-null bool
last10

In [18]:
df_train_data = pd.merge(df_train_data,df_train_diffTime, how='left',on='userid')

df_test_data = pd.merge(df_test_data,df_test_diffTime, how='left',on='userid')


In [19]:
df_train_data.fillna(-1,inplace=True)
df_test_data.fillna(-1,inplace=True)


In [20]:
# 最后k个时间差值的平均，k=[3,5,7,10,15]
df_train_data['last13DiffTimeAvg'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime']].mean(axis=1)

df_train_data['last15DiffTimeAvg'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime']].mean(axis=1)

df_train_data['last17DiffTimeAvg'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime']].mean(axis=1)

df_train_data['last110DiffTimeAvg'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime']].mean(axis=1)

df_train_data['last115DiffTimeAvg'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime',
                                                     'last13DiffTime','last14DiffTime','last15DiffTime']].mean(axis=1)


# 最后k个时间差值的平均，k=[3,5,7,10,15]
df_test_data['last13DiffTimeAvg'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime']].mean(axis=1)

df_test_data['last15DiffTimeAvg'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime']].mean(axis=1)

df_test_data['last17DiffTimeAvg'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime']].mean(axis=1)

df_test_data['last110DiffTimeAvg'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime']].mean(axis=1)


df_test_data['last115DiffTimeAvg'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime',
                                                     'last13DiffTime','last14DiffTime','last15DiffTime']].mean(axis=1)


In [21]:

################## 垃圾特征------------------------------------
# 最后k个时间差值的Var，k=[3,5,7,10]
df_train_data['last13DiffTimeVar'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime']].var(axis=1)

df_train_data['last15DiffTimeVar'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime']].var(axis=1)

df_train_data['last17DiffTimeVar'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime']].var(axis=1)

df_train_data['last110DiffTimeVar'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime']].var(axis=1)

df_train_data['last115DiffTimeVar'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime',
                                                     'last13DiffTime','last14DiffTime','last15DiffTime']].var(axis=1)


# 最后k个时间差值的Var，k=[3,5,7,10]
df_test_data['last13DiffTimeVar'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime']].var(axis=1)

df_test_data['last15DiffTimeVar'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime']].var(axis=1)

df_test_data['last17DiffTimeVar'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime']].var(axis=1)

df_test_data['last110DiffTimeVar'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime']].var(axis=1)

df_test_data['last115DiffTimeVar'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime',
                                                     'last13DiffTime','last14DiffTime','last15DiffTime']].var(axis=1)



In [22]:
################## 垃圾特征------------------------------------

# 最后k个时间差值的Std，k=[3,5,7,10]
df_train_data['last13DiffTimeStd'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime']].std(axis=1)

df_train_data['last15DiffTimeStd'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime']].std(axis=1)

df_train_data['last17DiffTimeStd'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime']].std(axis=1)

df_train_data['last110DiffTimeStd'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime']].std(axis=1)

df_train_data['last115DiffTimeStd'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime',
                                                     'last13DiffTime','last14DiffTime','last15DiffTime']].std(axis=1)


# 最后k个时间差值的Std，k=[3,5,7,10]
df_test_data['last13DiffTimeStd'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime']].std(axis=1)

df_test_data['last15DiffTimeStd'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime']].std(axis=1)

df_test_data['last17DiffTimeStd'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime']].std(axis=1)

df_test_data['last110DiffTimeStd'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime']].std(axis=1)
df_test_data['last115DiffTimeAvg'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime',
                                                     'last13DiffTime','last14DiffTime','last15DiffTime']].std(axis=1)



In [23]:
################## 垃圾特征------------------------------------

# 最后k个时间差值的Skew，k=[3,5,7,10]
df_train_data['last13DiffTimeSkew'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime']].skew(axis=1)

df_train_data['last15DiffTimeSkew'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime']].skew(axis=1)

df_train_data['last17DiffTimeSkew'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime']].skew(axis=1)

df_train_data['last110DiffTimeSkew'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime']].skew(axis=1)

# 最后k个时间差值的Skew，k=[3,5,7,10]
df_test_data['last13DiffTimeSkew'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime']].skew(axis=1)

df_test_data['last15DiffTimeSkew'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime']].skew(axis=1)

df_test_data['last17DiffTimeSkew'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime']].skew(axis=1)

df_test_data['last110DiffTimeSkew'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime']].skew(axis=1)



In [25]:
################## 垃圾特征------------------------------------

# 最后k个时间差值的Kurt，k=[5,7,10]
df_train_data['last15DiffTimeKurt'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime']].kurt(axis=1)

df_train_data['last17DiffTimeKurt'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime']].kurt(axis=1)

df_train_data['last110DiffTimeKurt'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime']].kurt(axis=1)

df_train_data['last115DiffTimeKurt'] = df_train_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime',
                                                     'last13DiffTime','last14DiffTime','last15DiffTime']].kurt(axis=1)


# 最后k个时间差值的Kurt，k=[5,7,10]
df_test_data['last15DiffTimeKurt'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime']].kurt(axis=1)

df_test_data['last17DiffTimeKurt'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime',
                                                    'last4DiffTime','last5DiffTime','last6DiffTime','last7DiffTime']].kurt(axis=1)

df_test_data['last110DiffTimeKurt'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime']].kurt(axis=1)
df_test_data['last115DiffTimeAvg'] = df_test_data[['last1DiffTime','last2DiffTime','last3DiffTime','last4DiffTime',
                                                     'last5DiffTime','last6DiffTime','last7DiffTime','last8DiffTime',
                                                     'last9DiffTime','last10DiffTime','last11DiffTime','last12DiffTime',
                                                     'last13DiffTime','last14DiffTime','last15DiffTime']].kurt(axis=1)



In [26]:
df_train_data.info()
# df_train_data.head()

df_test_data.info()
# df_test_data.head()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40307 entries, 0 to 40306
Data columns (total 67 columns):
userid                    40307 non-null int64
orderType                 40307 non-null int64
last1Type                 40307 non-null int64
last2Type                 40307 non-null int64
last3Type                 40307 non-null int64
last4Type                 40307 non-null int64
last5Type                 40307 non-null int64
last6Type                 40307 non-null int64
last7Type                 40307 non-null int64
last8Type                 40307 non-null int64
last9Type                 40307 non-null int64
last10Type                40307 non-null int64
last10IncludeType5        40307 non-null bool
last10IncludeType6        40307 non-null bool
last10IncludeType7        40307 non-null bool
last10IncludeType8        40307 non-null bool
last10IncludeType9        40307 non-null bool
last10IncludeType56       40307 non-null bool
last10IncludeType57       40307 non-null bool
last1

,userid,last1Type,last2Type,last3Type,last4Type,last5Type,last6Type,last7Type,last8Type,last9Type,...,last110DiffTimeStd,last115DiffTimeStd,last13DiffTimeSkew,last15DiffTimeSkew,last17DiffTimeSkew,last110DiffTimeSkew,last15DiffTimeKurt,last17DiffTimeKurt,last110DiffTimeKurt,last115DiffTimeKurt
0,100000000013,1,1,3,1,1,2,1,1,1,...,2.690780e+04,2.948346e+06,1.723442,-0.312377,0.182011,0.277203,-2.831957,-1.986438,-1.436001,14.949729
1,100000000111,1,5,6,0,0,0,0,0,0,...,7.515000e+06,6.135972e+06,1.732051,2.236068,2.645751,3.162278,5.000000,7.000000,10.000000,15.000000
2,100000000127,1,5,1,5,7,7,0,0,0,...,5.306034e+06,4.413053e+06,1.732042,0.645607,2.275716,2.758071,-3.106662,5.408040,7.952898,12.213909
3,100000000231,5,6,3,3,2,5,6,1,2,...,3.744782e+03,4.620358e+06,0.775726,-0.423616,1.591844,3.161239,-0.438312,2.969560,9.995154,14.999975
4,100000000379,6,2,1,5,6,2,1,5,6,...,1.857022e+04,2.542372e+06,-0.248942,2.235903,2.462303,2.963526,4.999409,6.179087,8.976834,14.977516


In [27]:
df_train_data.drop('orderType',axis=1,inplace=True)


In [28]:

df_train_data.to_csv('../data/df_train_1.csv',index=False,index_label=False)
df_test_data.to_csv('../data/df_test_1.csv',index=False,index_label=False)

